In [136]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.utils import np_utils
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization
import numpy as np

In [70]:
data = pd.read_csv("train.csv")
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [71]:
teste = pd.read_csv("test.csv")
teste.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
x_train = data.drop('label', axis=1)
y_train = data['label']

In [75]:
x_train = x_train.values.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
teste = teste.values.reshape(teste.shape[0], 28, 28, 1)
teste = teste.astype('float32')

In [78]:
x_train /= 255
teste /= 255

In [80]:
y_train = np_utils.to_categorical(y_train, 10)

In [90]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x_train, y_train, test_size=0.2)

In [151]:
classificador = Sequential()
#pré processamento da rede neural
classificador.add(Conv2D(32, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPool2D(pool_size=(2,2)))
#classificador.add(Flatten()) - em redes com mais de uma camada convolucional o flatten é aplicado apenas na ultima camada

classificador.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPool2D(pool_size=(2,2)))
classificador.add(Flatten())

#processamento da rede neural.
classificador.add(Dense(units=128, activation='relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units=128, activation='relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units=10, activation='softmax'))
#compile da rede
classificador.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#fit
classificador.fit(x_treino, y_treino, epochs=35, batch_size=128, validation_data=(x_teste,y_teste))

Epoch 1/35
263/263 [==============================] - 27s 101ms/step - loss: 0.5666 - accuracy: 0.8200 - val_loss: 0.8545 - val_accuracy: 0.7119
Epoch 2/35
263/263 [==============================] - 26s 100ms/step - loss: 0.0808 - accuracy: 0.9766 - val_loss: 0.0739 - val_accuracy: 0.9781
Epoch 3/35
263/263 [==============================] - 26s 100ms/step - loss: 0.0583 - accuracy: 0.9827 - val_loss: 0.0665 - val_accuracy: 0.9799
Epoch 4/35
263/263 [==============================] - 26s 100ms/step - loss: 0.0451 - accuracy: 0.9868 - val_loss: 0.0501 - val_accuracy: 0.9851
Epoch 5/35
263/263 [==============================] - 26s 100ms/step - loss: 0.0339 - accuracy: 0.9899 - val_loss: 0.0693 - val_accuracy: 0.9807
Epoch 6/35
263/263 [==============================] - 26s 101ms/step - loss: 0.0293 - accuracy: 0.9914 - val_loss: 0.0501 - val_accuracy: 0.9865
Epoch 7/35
263/263 [==============================] - 26s 100ms/step - loss: 0.0239 - accuracy: 0.9927 - val_loss: 0.0514 - val_ac

In [152]:
predicao = classificador.predict(teste)
predicao.shape

(28000, 10)

In [153]:
predicao = np.argmax(predicao, axis=1)
predicao = pd.Series(predicao)

In [154]:
sub = pd.concat([pd.Series(range(1,28001)), predicao], axis=1)
sub.columns = ['ImageId', 'Label']

In [132]:
sub

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [155]:
sub.to_csv("submissao_dig5.csv", index=False)